In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt

In [ ]:
turbine_pos_8M = np.array() # 좌표에 대한 2차원 행렬렬

# 0열 제거
turbine_pos_8M = np.delete(turbine_pos_8M, 0, axis=1)

In [3]:
turbine_pos_8M[:, 0:] /= 1000

In [ ]:
plt.scatter(turbine_pos_8M[:, 0], turbine_pos_8M[:, 1], color='C1', label='10MW')

for s in range(len(turbine_pos_8M)):
    plt.text(turbine_pos_8M[s, 0], turbine_pos_8M[s, 1], str(s+1))

plt.legend()
plt.title('Turbines')
plt.show()

In [ ]:
# 해상변전소 좌표 설정 (임의)
substations = []
sets = [(34, 35), (44, 45), (43, 44), (42, 43)] # 중점, 영역 안쪽 4개 substation
for (a, b) in sets:
    substations.append((turbine_pos_8M[a-1, :] + turbine_pos_8M[b-1, :])/2)\

substations = np.array(substations)
new_x = (turbine_pos_8M[44, 0]*3 - turbine_pos_8M[43, 0])/2
new_y = (turbine_pos_8M[44, 1]*3 - turbine_pos_8M[43, 1])/2
substations = np.insert(substations, 1, np.array([new_x, new_y]), axis=0)
temp = substations[1:, :].copy()
temp[:, 0] += 2
temp[:, 1] -= 0.5
substations_8M = np.vstack((substations, temp))


# 육지변전소 좌표 설정 (임의)
onshore_sub = np.array([substations_8M[5, 0]+16, turbine_pos_8M[48, 1]])

# substations 시각화
plt.figure(figsize=(15, 5))
plt.scatter(turbine_pos_8M[:, 0], turbine_pos_8M[:, 1], color='C0', label='10MW')
plt.scatter(substations_8M[:, 0], substations_8M[:, 1],  color='C1', label='10MW_OFFSUB')
plt.scatter(onshore_sub[0], onshore_sub[1], color='C2', label='10MW_ONSUB')

for s in range(len(turbine_pos_8M)):
    plt.text(turbine_pos_8M[s, 0], turbine_pos_8M[s, 1], str(s+1))

for itm, s in enumerate(substations_8M):
    plt.text(s[0], s[1], f'SUB_{itm+1}')
plt.text(onshore_sub[0], onshore_sub[1], 'ON_SUB')
plt.legend( bbox_to_anchor=(1.3, 0.5))
plt.title('Turbines with 9 substations')
plt.show()

In [6]:
nodes_8M = np.vstack((substations_8M, turbine_pos_8M)) # 상위 9개는 substation
num_nodes_8M = nodes_8M.shape[0]
num_sub_8M = substations_8M.shape[0]
power_production = {i:1 for i in range(9, num_nodes_8M)}
print(f'Number of 8MW turbines and substation : {num_nodes_8M}\nNumber of substations : {num_sub_8M}')

substations_index = list(range(9))
turbines_index = list(range(9,59))

Number of 8MW turbines and substation : 59
Number of substations : 9


In [7]:
model_8M = gp.Model('CableRoutingOptimization8M')

model_8M.setParam('TimeLimit', 3600)
model_8M.setParam('DisplayInterval', 20)
model_8M.setParam('Threads', 15)

cable_types = [1, 2, 3, 4, 5]
cable_capacity_8M = {1: 3.68, 2: 4.19, 3: 6.34, 4: 8.06, 5: 9} 
cable_cost = {1: 64, 2: 68, 3: 75, 4: 90, 5: 100} # Cost 다운 스케일링(1/1000)

off_on_cost = np.array([np.linalg.norm(substations_8M[i] - onshore_sub)* 180 for i in range(9)]) # 해상-육지 변전소 연결 비용

x = model_8M.addVars(num_nodes_8M, num_nodes_8M, cable_types, vtype=GRB.BINARY, name='x')
y = model_8M.addVars(num_nodes_8M, num_nodes_8M, vtype=GRB.BINARY, name='y')
f = model_8M.addVars(num_nodes_8M, num_nodes_8M, vtype=GRB.CONTINUOUS, lb=0, name='f')
u = model_8M.addVars(num_sub_8M, vtype=GRB.BINARY, name='n')

# 목적함수
# 식(1)
model_8M.setObjective(gp.quicksum(cable_cost[t] * np.linalg.norm(nodes_8M[i]-nodes_8M[j], ord=2) * x[i, j, t] 
                                  for i in range(num_nodes_8M) 
                                  for j in range(num_nodes_8M) 
                                  for t in cable_types if i!=j)\
                      + gp.quicksum(off_on_cost[k] * u[k] for k in range(num_sub_8M)), GRB.MINIMIZE)

# 제약조건
# 식(2)
for i in range(num_nodes_8M):
    for j in range(num_nodes_8M):
        if i!=j:
            model_8M.addConstr(gp.quicksum(x[i, j, t] for t in cable_types) == y[i, j], 
                               name=f'cable_type_selection_{i}_{j}')

# 식(3)
for h in range(9, num_nodes_8M):
    model_8M.addConstr(gp.quicksum(f[h, i] - f[i, h] for i in range(num_nodes_8M) if i!=h) == power_production[h], 
                       name=f'flow_conservation_{h}')

# 식(4)
for i in range(num_nodes_8M):
     for j in range(num_nodes_8M):
          if i!=j:
               model_8M.addConstr(gp.quicksum(cable_capacity_8M[t] * x[i, j, t] for t in cable_types) >= f[i, j], 
                                  name=f'capacity_constraint_{i}_{j}')

# 식(5)
for h in range(9, num_nodes_8M):
    model_8M.addConstr(gp.quicksum(y[h, j] for j in range(num_nodes_8M) if j!=h) == 1, name=f'one_outgoing_cable_{h}')

# 식(6)
for h in range(num_sub_8M):
    model_8M.addConstr(gp.quicksum(y[h, j] for j in range(num_nodes_8M) if j!=h) == 0, name='no_outgoing_cable_substation')

# 해상변전소 제약조건
model_8M.addConstr(gp.quicksum(u[i] for i in range(num_sub_8M)) == 1, name='one_offshore_substation')

# 해상변전소 제약조건 (2)
for i in range(num_nodes_8M):
    for j in range(num_sub_8M):
        if i!=j:
            model_8M.addConstr(y[i, j] <= u[j], 'one_incomming_substation')

model_8M.optimize()

Set parameter TimeLimit to value 3600
Set parameter DisplayInterval to value 20
Set parameter Threads to value 15
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-13600KF, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 15 threads

Optimize a model with 7476 rows, 24376 columns and 51339 nonzeros
Model fingerprint: 0x2ad43890
Variable types: 3481 continuous, 20895 integer (20895 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [6e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1825 rows and 4767 columns
Presolve time: 0.12s
Presolved: 5651 rows, 19609 columns, 42559 nonzeros
Variable types: 2900 continuous, 16709 integer (16709 binary)
Found heuristic solution: objective 18445.793105

Root relaxation: objective 7.461179e+03, 8686 iterations, 0.14 seconds (0.42 work units)

    

In [8]:
print(f'Optimal solution found with total cost : {model_8M.objVal}')
print(f'Number of decision variables : {model_8M.NumVars}')
print(f'Number of constraints : {model_8M.NumConstrs}')

connections = []
for i in range(num_nodes_8M):
    for j in range(num_nodes_8M):
        if i != j and y[i, j].x > 0.5:
            for t in cable_types:
                if x[i, j, t].x > 0.5:
                    connections.append((i, j, t))

Optimal solution found with total cost : 8160.260370235466
Number of decision variables : 24376
Number of constraints : 7476


In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(turbine_pos_8M[:, 0], turbine_pos_8M[:, 1], color='C0', label='8MW')
plt.scatter(substations_8M[:, 0], substations_8M[:, 1],  color='C1', label='8MW_OFFSUB')
plt.scatter(onshore_sub[0], onshore_sub[1], color='C2', label='8MW_ONSUB')

for s in range(len(turbine_pos_8M)):
    plt.text(turbine_pos_8M[s, 0], turbine_pos_8M[s, 1], str(s+1))

for itm, s in enumerate(substations_8M):
    plt.text(s[0], s[1], f'SUB_{itm+1}')
plt.text(onshore_sub[0], onshore_sub[1], 'ON_SUB')
cable_colors = {
    1: 'blue',
    2: 'green',
    3: 'orange',
    4: 'purple',
    5: 'red'
}

for (i, j, t) in connections:
    plt.plot([nodes_8M[i, 0], nodes_8M[j, 0]], [nodes_8M[i, 1], nodes_8M[j, 1]], color=cable_colors[t], label=f'cable_type_{t}')

plt.plot([nodes_8M[3, 0], onshore_sub[0]], [nodes_8M[3, 1], onshore_sub[1]], color = 'cyan', label='cable_type_6')

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(1, 1))

plt.title('Optimized connections')
plt.show()    

In [10]:
connections

[(9, 14, 1),
 (10, 15, 1),
 (11, 16, 1),
 (12, 17, 1),
 (13, 18, 1),
 (14, 19, 1),
 (15, 20, 1),
 (16, 21, 1),
 (17, 22, 1),
 (18, 23, 1),
 (19, 24, 1),
 (20, 25, 1),
 (21, 26, 1),
 (22, 27, 1),
 (23, 28, 1),
 (24, 29, 2),
 (25, 30, 2),
 (26, 31, 2),
 (27, 32, 2),
 (28, 33, 2),
 (29, 34, 3),
 (30, 35, 3),
 (31, 36, 3),
 (32, 37, 3),
 (33, 38, 3),
 (34, 39, 3),
 (35, 40, 3),
 (36, 41, 3),
 (37, 42, 3),
 (38, 43, 3),
 (39, 45, 4),
 (40, 46, 4),
 (41, 3, 4),
 (42, 47, 4),
 (43, 48, 4),
 (44, 49, 1),
 (45, 51, 4),
 (46, 3, 4),
 (47, 3, 4),
 (48, 52, 4),
 (49, 54, 1),
 (50, 55, 1),
 (51, 3, 5),
 (52, 3, 5),
 (53, 58, 1),
 (54, 55, 1),
 (55, 56, 3),
 (56, 3, 3),
 (57, 3, 1),
 (58, 57, 1)]